In [97]:
import sys
import os
import numpy as np
import numpy.matlib
import cv2
#print('\ntextdetection.py')
#print('       A demo script of the Extremal Region Filter algorithm described in:')
#print('       Neumann L., Matas J.: Real-Time Scene Text Localization and Recognition, CVPR 2012\n')
path_to_image_to_be_processed = '/home/pboord/Downloads/yelp/waitrose.jpg'
#pathname = os.path.dirname(sys.argv[0])
pathname = '/home/pboord/Downloads/yelp'
img      = cv2.imread(path_to_image_to_be_processed)
# for visualization
vis      = img.copy()
#print cv2.__version__
# Extract channels to be processed individually
channels = cv2.text.computeNMChannels(img)
# Append negative channels to detect ER- (bright regions over dark background)
cn = len(channels)-1
for c in range(0,cn):
  channels.append((255-channels[c]))
# Apply the default cascade classifier to each independent channel (could be done in parallel)
#print("Extracting Class Specific Extremal Regions from "+str(len(channels))+" channels ...")
#print("    (...) this may take a while (...)")
def overlap(b1, b2):
    return (b1[0] <= b2[0]+b2[2]) and (b1[0]+b1[2] >= b2[0]) and (b1[1] <= b2[1]+b2[3]) and (b1[1]+b1[3] >= b2[1])
def bigrect(b1, b2):
    b0 = b1
    b0[0]=min(b1[0],b2[0])
    b0[1]=min(b1[1],b2[1])
    b0[2]=max(b1[0]+b1[2],b2[0]+b2[2]) - b0[0]
    b0[3]=max(b1[1]+b1[3],b2[1]+b2[3]) - b0[1]
    return b0
def lbwh2lbrt(b):
    b0=b
    b0[0]=b[0]
    b0[1]=b[1]
    b0[2]=b[0]+b[2]
    b0[3]=b[1]+b[3]
    return b0
b = []
for channel in channels:

  erc1 = cv2.text.loadClassifierNM1(pathname+'/trained_classifierNM1.xml')
  er1 = cv2.text.createERFilterNM1(erc1,16,0.00015,0.13,0.2,True,0.1)

  erc2 = cv2.text.loadClassifierNM2(pathname+'/trained_classifierNM2.xml')
  er2 = cv2.text.createERFilterNM2(erc2,0.5)
    # detectRegions: function in MSER class:
    # http://docs.opencv.org/3.0.0/d3/d28/classcv_1_1MSER.html
    # https://en.wikipedia.org/wiki/Maximally_stable_extremal_regions
  regions = cv2.text.detectRegions(channel,er1,er2)
  #print regions
    # erGrouping:
    # Find groups of Extremal Regions that are organized as text blocks
  rects = cv2.text.erGrouping(img,channel,[r.tolist() for r in regions])
  #rects = cv2.text.erGrouping(img,gray,[x.tolist() for x in regions], cv2.text.ERGROUPING_ORIENTATION_ANY,'../../GSoC2014/opencv_contrib/modules/text/samples/trained_classifier_erGrouping.xml',0.5)
  #print rects
  #Visualization
  for r in range(0,np.shape(rects)[0]):
    b.append(rects[r])
# consolidate overlapping rectangles
c = b
h=0
while h < len(c):
    g=len(c) - 1
    while g > h:
        if overlap(c[g],c[h]):
            c[g] = bigrect(c[g],c[h])
            c = np.delete(c,h,axis=0)
        g -= 1
    h += 1
h=0
while h < len(c):
    g=len(c) - 1
    while g > h:
        if overlap(c[g],c[h]):
            c[g] = bigrect(c[g],c[h])
            c = np.delete(c,h,axis=0)
        g -= 1
    h += 1

In [146]:
#
x=400
y=20
# # right numberplate
# x = 658
# y = 369
# # back light
# x = 533
# y = 376

# center of mass (cm)
rectCM = np.zeros((c.shape[0],2), dtype=int)
rectCM[:,0] = c[:,0] + c[:,2]/2
rectCM[:,1] = c[:,1] + c[:,3]/2
cmDiff = rectCM - np.matlib.repmat(np.array([x,y]),3,1)
closestRect = np.argmin(np.hypot(cmDiff[:,0],cmDiff[:,1]))

In [147]:
#print rect
rect = c[closestRect,:]
cv2.rectangle(vis, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (0, 0, 0), 2)
cv2.rectangle(vis, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (255, 255, 255), 1)
cv2.imwrite('waitrose_boxed.jpg',vis)

True

In [148]:
imSeg = img[ rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2] ]
cv2.imwrite('textbox.jpg',imSeg)
# for i in range(0,len(c)):
#     #Visualization
#     cv2.imshow("Text detection result", imSeg)
#     cv2.waitKey(2000)

True

In [149]:
txt = tool.image_to_string(
    Image.open('textbox.jpg'),
    lang="eng",
    builder=pyocr.builders.TextBuilder()
)
txt

u'Waitrose'

In [130]:
a=Image.open('textbox.jpg')

In [131]:
type(a)

PIL.JpegImagePlugin.JpegImageFile

In [132]:
type(imSeg)

numpy.ndarray

In [133]:
b=open('/home/pboord/Downloads/yelp/textbox.jpg', 'rb').read().encode('base64').replace('\n', '')

In [134]:
type(b)

str

In [104]:
from PIL import Image
import sys

import pyocr
import pyocr.builders

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)
# The tools are returned in the recommended order of usage
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'

langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))
# Ex: Will use lang 'fra'
# Note that languages are NOT sorted in any way. Please refer
# to the system locale settings for the default language
# to use.

Will use tool 'Tesseract (sh)'
Available languages: Info in bmfCreate: Generating pixa of bitmap fonts from string, osd, eng, equ
Will use lang 'Info in bmfCreate: Generating pixa of bitmap fonts from string'


In [135]:
txt = tool.image_to_string(
    Image.open('textbox.jpg'),
    lang="eng",
    builder=pyocr.builders.TextBuilder()
)
txt

u'Waitrose'

In [150]:
txt = tool.image_to_string(
    Image.fromarray(imSeg),
    lang="eng",
    builder=pyocr.builders.TextBuilder()
)
txt

u'Waitrose'

In [116]:
word_boxes = tool.image_to_string(
    Image.open('textbox.jpg'),
    lang="eng",
    builder=pyocr.builders.WordBoxBuilder()
)

In [122]:
[method for method in dir(word_boxes) if callable(getattr(word_boxes, method))]

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [106]:
!tesseract textbox.jpg out

Info in bmfCreate: Generating pixa of bitmap fonts from string
Tesseract Open Source OCR Engine vbd45b3a with Leptonica
Warning. Invalid resolution 0 dpi. Using 70 instead.


In [94]:
from google.cloud import vision
client = vision.Client()
with open('./im_1.jpg', 'rb') as image_file:
    image = client.image(content=image_file.read())
    texts = image.detect_text()
    texts[0].locale
    texts[0].description
    texts[1].description

ImportError: No module named google.cloud

In [9]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('digits.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

# Initiate kNN, train the data, then test it with test data for k=1
knn = cv2.KNearest()
knn.train(train,train_labels)
ret,result,neighbours,dist = knn.find_nearest(test,k=5)

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print accuracy

error: /home/travis/miniconda/conda-bld/conda_1485299288502/work/opencv-3.2.0/modules/imgproc/src/color.cpp:9748: error: (-215) scn == 3 || scn == 4 in function cvtColor
